# OntoGPT graph generation
We ran OntoGPT on the desiccation subset of our dataset; here, we'll post-process the output and format it as a graph.

In [1]:
from ontogpt.io.csv_wrapper import parse_yaml_predictions

/mnt/home/lotrecks/anaconda3/envs/ontogpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read in OntoGPT output